In [2]:
import json
import base64
import requests
from PIL import Image
from datetime import datetime
from cryptography.fernet import Fernet

def generate_key():
    return Fernet.generate_key()

# Encrypt a message using symmetric encryption
def encrypt_message(message, key):
    cipher = Fernet(key)
    return cipher.encrypt(message.encode())

def decrypt_message(ciphertext, key):
    cipher = Fernet(key)
    return cipher.decrypt(ciphertext).decode()

def get_current_location():
    response = requests.get('https://ipinfo.io/json')
    data = response.json()
    city = data.get('city', '')
    region = data.get('region', '')
    country = data.get('country', '')
    loc = data.get('loc', '')  # latitude,longitude
    return {
        'city': city,
        'region': region,
        'country': country,
        'coordinates': loc
    }

## Encoder

In [3]:
def encode_image(img_path, secret_message, key, output_path):
    # Encrypt message first
    enc_bytes = encrypt_message(secret_message, key)
    # For embedding, encode bytes to base64 string (to ensure printable chars)
    enc_b64 = base64.urlsafe_b64encode(enc_bytes).decode()
    # Add a null-terminator for extraction
    enc_b64 += chr(0)

    img = Image.open(img_path)
    encoded = img.copy()
    width, height = img.size
    data = ''.join(f"{ord(c):08b}" for c in enc_b64)
    idx = 0

    for y in range(height):
        for x in range(width):
            pixel = list(img.getpixel((x, y)))
            for n in range(3):  # R, G, B
                if idx < len(data):
                    pixel[n] = pixel[n] & ~1 | int(data[idx])
                    idx += 1
            encoded.putpixel((x, y), tuple(pixel))
            if idx >= len(data):
                encoded.save(output_path)
                return

## Decoder

In [4]:
def decode_image(img_path, key):
    img = Image.open(img_path)
    width, height = img.size
    bits = []
    for y in range(height):
        for x in range(width):
            pixel = img.getpixel((x, y))
            for n in range(3):
                bits.append(str(pixel[n] & 1))
    chars = [chr(int(''.join(bits[i:i+8]), 2)) for i in range(0, len(bits), 8)]
    message = ''.join(chars)
    # Extract message up to null-terminator
    enc_b64 = message.split(chr(0))[0]
    # Decode base64 to bytes
    enc_bytes = base64.urlsafe_b64decode(enc_b64)
    # Decrypt
    return decrypt_message(enc_bytes, key)

## Usage example

In [5]:
inputPath = "raw.png"
outputPath = "encoded.png"

secret_data = {
    "message": "Very Secret Message",
    "date": datetime.now().isoformat(),
    "location": get_current_location()
}

secret_message = json.dumps(secret_data, indent=4)

key = generate_key()
print("Generated encryption key (store this securely):")
print(key.decode())

# To encode:
encode_image(inputPath, secret_message, key, outputPath)

# To decode:
message = decode_image(outputPath, key)
print("\nDecoded message:")
print(secret_message)

Generated encryption key (store this securely):
ZN_QWDfohUGE2ZJW5gYHZgegnC2AblUbalLvnLBpEXY=

Decoded message:
{
    "message": "Very Secret Message",
    "date": "2025-06-15T08:49:04.500639",
    "location": {
        "city": "Plessisville",
        "region": "Quebec",
        "country": "CA",
        "coordinates": "46.2186,-71.7620"
    }
}
